<a href="https://colab.research.google.com/github/AV-Gorshkov/Python_netology/blob/main/PY_Pandas_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# импорт библиотек

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split


from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
# Загружаем данные в датафрейм

param = r'/content/drive/My Drive/Нетология/Аналитика_Big_Data/дз_итог/'

file_1 = 'user.csv'
file_2 = 'item.csv'
file_3 = 'data.csv'

user_column = ['user_id', 'age', 'gender', 'occupation','zip_code']
df_user = pd.read_csv( f'{param}{file_1}' , sep = '|',  header=None, names=user_column)

item_column = ['film_id', 'movie_title', 'release_date', 'video_release_date',\
               'URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children',\
               'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',\
               'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',\
               'War', 'Western']
df_movie = pd.read_csv( f'{param}{file_2}',  sep='|', encoding='latin-1', header=None, names=item_column)

data_column = ['user_id', 'film_id', 'rating', 'timestamp']
df_rating = pd.read_csv( f'{param}{file_3}' , sep = '\t',  header=None, names=data_column)

In [3]:
# Задание_1
# Загрузите в колаб файлы по оценкам и фильмам и создайте на их основе pandas-датафреймы

# датафрем-фильмы
df_movie.head()

,film_id,movie_title,release_date,video_release_date,URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [4]:
# датафрем-рейтинг
df_rating.head()

,user_id,film_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [5]:
# Задание_2
# Найдите id пользователя, поставившего больше всего оценок

df_user_rating = df_rating.groupby('user_id').agg({'rating': 'count'}).reset_index(). sort_values('rating', ascending=False)

df_user_rating.head(1)

,user_id,rating
404,405,737


In [6]:
#  Вариант_2

df_user_rating[ df_user_rating['rating'] == df_user_rating['rating'].max()]

,user_id,rating
404,405,737


In [7]:
# Задание_3
# Оставьте в датафрейме ratings только те фильмы, который оценил данный пользователь

idx = df_user_rating[ df_user_rating['rating'] == df_user_rating['rating'].max()].index  # индекс строки с Max значением рейтинга
id_user = df_user_rating.at[ idx[0], 'user_id']                                             # значение пользователя по Max индексу

df_user_max = df_rating[df_rating['user_id'] == id_user]

df_user_film = df_user_max.merge( df_movie, how = 'left', left_on='film_id', right_on='film_id')[[ 'user_id', 'film_id', 'rating', 'movie_title' ]]

df_user_film.head()

,user_id,film_id,rating,movie_title
0,405,56,4,Pulp Fiction (1994)
1,405,592,1,True Crime (1995)
2,405,1582,1,T-Men (1947)
3,405,171,1,Delicatessen (1991)
4,405,580,1,"Englishman Who Went Up a Hill, But Came Down a..."


In [8]:
# Задание_4
# Добавьте к датафрейму из задания 3 столбцы:
# по жанрам / столбцы с общим количеством оценок от всех пользователей на фильм / столбец суммарной оценкой от всех пользователей.

# создадим датафрейм с общим количеством оценок от всех пользователей на фильм и суммарной оценкой от всех пользователей

df_film_rating = df_rating.groupby('film_id').agg({'rating': ['sum', 'count']}).reset_index().set_axis( ['film_id', 'sum_rating','count_rating'], axis=1)

df_film_rating.head()

,film_id,sum_rating,count_rating
0,1,1753,452
1,2,420,131
2,3,273,90
3,4,742,209
4,5,284,86


In [9]:
# объеденим датафрейм из Задания_3 с датафреймом по оценкам фильма

df_film_user = df_user_max.merge( df_film_rating, how = 'left', left_on='film_id', right_on='film_id')

# добаыим информацию по жанрам фильмов и дате фильма

name_column = ['user_id', 'film_id', 'rating','sum_rating',	'count_rating', 'release_date',\
               'unknown', 'Action', 'Adventure', 'Animation', 'Children',\
               'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',\
               'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',\
               'War', 'Western']

df_film_user = df_film_user.merge( df_movie, how = 'left', left_on='film_id', right_on='film_id' ). loc[ : , name_column]

# df_film_user = df_film_user.loc[ : , df_film_user.columns.isin(name_column) ]

df_film_user.head()

,user_id,film_id,rating,sum_rating,count_rating,release_date,unknown,Action,Adventure,Animation,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,405,56,4,1600,394,01-Jan-1994,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,405,592,1,30,9,01-Jan-1995,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,405,1582,1,1,1,01-Jan-1947,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,405,171,1,252,65,01-Jan-1991,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,405,580,1,108,32,01-Jan-1995,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [10]:
# Задание_5
#  Сформировать выборки X_train, X_test, y_train, y_test

#  изучим дата фрейм для модели ML
df_model = df_film_user.copy()

df_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 737 entries, 0 to 736
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       737 non-null    int64 
 1   film_id       737 non-null    int64 
 2   rating        737 non-null    int64 
 3   sum_rating    737 non-null    int64 
 4   count_rating  737 non-null    int64 
 5   release_date  737 non-null    object
 6   unknown       737 non-null    int64 
 7   Action        737 non-null    int64 
 8   Adventure     737 non-null    int64 
 9   Animation     737 non-null    int64 
 10  Children      737 non-null    int64 
 11  Comedy        737 non-null    int64 
 12  Crime         737 non-null    int64 
 13  Documentary   737 non-null    int64 
 14  Drama         737 non-null    int64 
 15  Fantasy       737 non-null    int64 
 16  Film-Noir     737 non-null    int64 
 17  Horror        737 non-null    int64 
 18  Musical       737 non-null    int64 
 19  Mystery 

In [11]:
# Для построения модели сформируем признаки:
# Год выхода / Жанры / Общее количество оценок / Суммарную оценку / жанр фильма

# в датафрейм добавим признак - год фильма

df_model['year'] = df_model['release_date'].str[-4:].astype(int)

# сформируем датафрейм с признаками и целевой переменной

name_column_model = ['year', 'sum_rating', 'count_rating',\
               'unknown', 'Action', 'Adventure', 'Animation', 'Children',\
               'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',\
               'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',\
               'War', 'Western']

df_model = df_model. loc[ : , name_column_model]
data_target = pd.array(df_film_user['rating'])

df_model.head()
# data_target

,year,sum_rating,count_rating,unknown,Action,Adventure,Animation,Children,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1994,1600,394,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1995,30,9,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,1947,1,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1991,252,65,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,1995,108,32,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [12]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 737 entries, 0 to 736
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   year          737 non-null    int64
 1   sum_rating    737 non-null    int64
 2   count_rating  737 non-null    int64
 3   unknown       737 non-null    int64
 4   Action        737 non-null    int64
 5   Adventure     737 non-null    int64
 6   Animation     737 non-null    int64
 7   Children      737 non-null    int64
 8   Comedy        737 non-null    int64
 9   Crime         737 non-null    int64
 10  Documentary   737 non-null    int64
 11  Drama         737 non-null    int64
 12  Fantasy       737 non-null    int64
 13  Film-Noir     737 non-null    int64
 14  Horror        737 non-null    int64
 15  Musical       737 non-null    int64
 16  Mystery       737 non-null    int64
 17  Romance       737 non-null    int64
 18  Sci-Fi        737 non-null    int64
 19  Thriller      737 non-null   

In [13]:
# приведем данные к единому масштабу
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

df_scaled = scaler.fit_transform(df_model)
df_scaled

array([[ 0.47114156,  3.93674974,  3.81558486, ..., -0.3939193 ,
        -0.23637666, -0.17125889],
       [ 0.549066  , -0.6799929 , -0.74264754, ...,  2.53859104,
        -0.23637666, -0.17125889],
       [-3.19130736, -0.76527031, -0.83736405, ..., -0.3939193 ,
        -0.23637666, -0.17125889],
       ...,
       [ 0.549066  , -0.7358643 , -0.76632667, ..., -0.3939193 ,
        -0.23637666, -0.17125889],
       [ 0.08151933,  2.12239929,  2.12252711, ..., -0.3939193 ,
        -0.23637666, -0.17125889],
       [ 0.47114156, -0.61235909, -0.68344971, ..., -0.3939193 ,
        -0.23637666, -0.17125889]])

In [14]:
# разделяем выборку на тренировочную и тестовую

x_train, x_test, y_train, y_test = train_test_split(df_scaled, data_target, train_size=0.2)

In [17]:
# Задание_6
# Создать модель линейной регрессии и обучите ее
lr = LinearRegression()

lr.fit(x_train, y_train)

print((f'коэфф модели:\n{lr.coef_}'))

коэфф модели:
[-5.01477269e-02 -4.17318117e-01  1.15819416e+00 -2.10942375e-15
 -1.29442773e-01 -1.83873777e-01 -4.79588724e-02 -4.52273612e-02
  5.06207104e-02 -3.92711627e-02 -1.15308758e-01 -1.36361034e-01
  9.36230351e-02 -6.68860952e-02 -1.26485240e-01 -1.42661279e-02
  1.92220568e-01 -7.06755171e-02 -9.66086064e-02  2.18212840e-02
  3.24686276e-02 -1.12815806e-01]


In [18]:
# Задание_7
# Оценить качество модели
y_pred = lr.predict(x_test)

# точность модели при помощи MSE

print( 'для выборки "Train" MSE =', mean_squared_error(y_train, lr.predict(x_train) ) )
print( 'для выборки "Text" MSE =', mean_squared_error(y_test, y_pred) )

для выборки "Train" MSE = 1.0917602841755008
для выборки "Text" MSE = 1.4020127249307035


In [16]:
# Модель логистической регрессии
logreg = LogisticRegression()
logreg.fit(x_train, y_train)

y_pred_logreg = logreg.predict(x_test)

# точность модели
print( 'accuracy', accuracy_score(y_test, y_pred_logreg ) )
print( 'f1', f1_score(y_test, y_pred_logreg, average='weighted' ) )

accuracy 0.6389830508474577
f1 0.5760586060558724


Точность модели LR accuracy = 0,63 значение F-мера = 0,57. На выходе получили среднюю модель для предсказания событий.

In [ ]:
# Задание_8
# Загрузить данные в spark

# установим PySpark

!apt-get update

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://downloads.apache.org/spark/spark-3.4.2/spark-3.4.2-bin-hadoop3.tgz

!tar -xvf spark-3.4.2-bin-hadoop3.tgz

!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy/main Sources [2,247 kB]
Get:12 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy/main amd64 Packages [1,153 kB]
Fetched 3,651 kB in 2s (

In [ ]:
# импорт библиотек для Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.2-bin-hadoop3"

import findspark
findspark.init()
from pyspark.sql import SparkSession

from pyspark.sql.types import StructType,StructField, StringType, IntegerType , BooleanType
from pyspark.sql import functions as f

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
# загружаем данные из файлов

item_shema = StructType() \
            .add( "film_id",IntegerType(), True) \
            .add( "movie_title", StringType(), True) \
            .add( "release_date", StringType(), True) \
            .add( "video_release_date", StringType(), True) \
            .add( "URL", StringType(), True) \
            .add( "unknown", IntegerType(), True) \
            .add( "Action",IntegerType(), True) \
            .add( "Adventure",IntegerType(), True) \
            .add( "Animation",IntegerType(), True) \
            .add( "Children",IntegerType(), True) \
            .add( "Comedy",IntegerType(), True) \
            .add( "Crime",IntegerType(), True) \
            .add( "Documentary",IntegerType(), True) \
            .add( "Drama",IntegerType(), True) \
            .add( "Fantasy",IntegerType(), True) \
            .add( "Film-Noir",IntegerType(), True) \
            .add( "Horror",IntegerType(), True) \
            .add( "Musical",IntegerType(), True) \
            .add( "Mystery",IntegerType(), True) \
            .add( "Romance",IntegerType(), True) \
            .add( "Sci-Fi",IntegerType(), True) \
            .add( "Thriller",IntegerType(), True) \
            .add( "War",IntegerType(), True) \
            .add( "Western",IntegerType(), True)

spark_movie = spark.read.csv( f'{param}{file_2}',  sep='|', header=False, schema=item_shema)


data_schema =  StructType() \
            .add( "user_id",IntegerType(), True) \
            .add( "film_id",IntegerType(), True) \
            .add( "rating",IntegerType(), True) \
            .add( "timestamp",IntegerType(), True) \

spark_rating = spark.read.csv( f'{param}{file_3}' , sep = '\t', schema=data_schema,  header=False)

spark_movie.show(7)

+-------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|film_id|         movie_title|release_date|video_release_date|                 URL|unknown|Action|Adventure|Animation|Children|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+-------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|      1|    Toy Story (1995)| 01-Jan-1995|              null|http://us.imdb.co...|      0|     0|        0|        1|       1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|      2|    GoldenEye (1995)| 01-Jan-1995|             

In [ ]:
spark_movie.schema

StructType([StructField('film_id', IntegerType(), True), StructField('movie_title', StringType(), True), StructField('release_date', StringType(), True), StructField('video_release_date', StringType(), True), StructField('URL', StringType(), True), StructField('unknown', IntegerType(), True), StructField('Action', IntegerType(), True), StructField('Adventure', IntegerType(), True), StructField('Animation', IntegerType(), True), StructField('Children', IntegerType(), True), StructField('Comedy', IntegerType(), True), StructField('Crime', IntegerType(), True), StructField('Documentary', IntegerType(), True), StructField('Drama', IntegerType(), True), StructField('Fantasy', IntegerType(), True), StructField('Film-Noir', IntegerType(), True), StructField('Horror', IntegerType(), True), StructField('Musical', IntegerType(), True), StructField('Mystery', IntegerType(), True), StructField('Romance', IntegerType(), True), StructField('Sci-Fi', IntegerType(), True), StructField('Thriller', Inte

In [ ]:
spark_rating.show(7)

+-------+-------+------+---------+
|user_id|film_id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
|    298|    474|     4|884182806|
|    115|    265|     2|881171488|
+-------+-------+------+---------+
only showing top 7 rows



In [ ]:
# Задание_9
# Spark - вывести среднюю оценку для каждого фильма

df_film_rating = spark_rating.groupBy('film_id').avg('rating')

df_film_rating.show(7)

+-------+------------------+
|film_id|       avg(rating)|
+-------+------------------+
|    496| 4.121212121212121|
|    471|3.6108597285067874|
|    463| 3.859154929577465|
|    148|          3.203125|
|   1342|               2.5|
|    833| 3.204081632653061|
|   1088| 2.230769230769231|
+-------+------------------+
only showing top 7 rows



In [ ]:
# Задание_10
# Spark - среднюю оценку для каждого жанра

# добавим рейтинг фильмов из df_film_rating в spark_movie

spark_movie_rating = spark_movie.join( df_film_rating, df_film_rating['film_id'] == spark_movie['film_id'], "left" )

name_col =  ['avg(rating)',\
               'unknown', 'Action', 'Adventure', 'Animation', 'Children',\
               'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',\
               'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',\
               'War', 'Western']

spark_movie_rating = spark_movie_rating.select(name_col)

spark_movie_rating.show()

+------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|       avg(rating)|unknown|Action|Adventure|Animation|Children|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|3.8783185840707963|      0|     0|        0|        1|       1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|3.2061068702290076|      0|     1|        1|        0|       0|     0|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       1|  0|      0|
| 3.033333333333333|      0|     0|        0|        0|       0|     0|    0|          0|    0|      0|        0|     0|      0|      0

In [ ]:
# Вариант_1
# функция melt для Spark (из интернета)

from pyspark.sql.functions import array, col, explode, lit, struct
from pyspark.sql import DataFrame
from typing import Iterable

def melt(
        df: DataFrame,
        id_vars: Iterable[str], value_vars: Iterable[str],
        var_name: str="variable", value_name: str="value") -> DataFrame:

    # Create array<struct<variable: str, value: ...>>
    _vars_and_vals = array(*(
        struct(lit(c).alias(var_name), col(c).alias(value_name))
        for c in value_vars))

    # Add to the DataFrame and explode
    _tmp = df.withColumn("_vars_and_vals", explode(_vars_and_vals))

    cols = id_vars + [
            col("_vars_and_vals")[x].alias(x) for x in [var_name, value_name]]
    return _tmp.select(*cols)

In [ ]:
lst_name =  [  'unknown', 'Action', 'Adventure', 'Animation', 'Children',\
               'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',\
               'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',\
               'War', 'Western']

sp_rating = melt(spark_movie_rating, id_vars=['avg(rating)'], value_vars=lst_name).withColumnRenamed('variable', 'genre')

sp_rating = sp_rating.withColumn('avg_rating', sp_rating['avg(rating)'] * sp_rating['value']  )

sp_rating = sp_rating.filter( sp_rating[ 'avg_rating'] != 0)

sp_rating = sp_rating.groupBy('genre').avg('avg_rating').sort( 'genre',ascending = True)

sp_rating.show()

+-----------+------------------+
|      genre|   avg(avg_rating)|
+-----------+------------------+
|     Action| 2.966332403758986|
|  Adventure|  3.14396731369761|
|  Animation| 3.298813080382377|
|   Children| 2.916884891868898|
|     Comedy|3.0005649618406713|
|      Crime| 3.211014749599754|
|Documentary| 3.229273094093941|
|      Drama|3.1873534142973337|
|    Fantasy| 2.849830297920943|
|  Film-Noir|3.5483508757184246|
|     Horror| 2.730157352996138|
|    Musical|3.3764231235398725|
|    Mystery|3.3368139323830235|
|    Romance|3.2440490301648004|
|     Sci-Fi| 3.165446053251289|
|   Thriller| 3.136692475563835|
|        War| 3.489185428943571|
|    Western| 3.185617473056144|
|    unknown|2.2222222222222223|
+-----------+------------------+



In [ ]:
# Вариант_2 - собственная фун-ция

name_lst = [ x for x in spark_movie_rating.schema.names if x !='avg(rating)'] # список столбцов датафрема Spark

def func_rating(data, name_lst):

  df = pd.DataFrame()

  for name in name_lst:

    total = 0
    count_el = 0
    for el in data.collect():

      _rating = el['avg(rating)'] * el[name]
      if _rating != 0:
        total += _rating
        count_el += 1

    row = { 'genre': name, 'avg_rating': total / count_el}

    df = pd.concat( [df, pd.DataFrame([row]) ] )  # собираем данные построно в датафрейм Pandas

  return spark.createDataFrame(df)                # датафрейм переносим в Spark

sp_rating_2 = func_rating(spark_movie_rating, name_lst)

sp_rating_2.sort( 'genre',ascending = True).show()

+-----------+------------------+
|      genre|        avg_rating|
+-----------+------------------+
|     Action| 2.966332403758986|
|  Adventure|3.1439673136976105|
|  Animation|3.2988130803823776|
|   Children| 2.916884891868897|
|     Comedy|3.0005649618406727|
|      Crime|3.2110147495997547|
|Documentary| 3.229273094093941|
|      Drama|3.1873534142973314|
|    Fantasy| 2.849830297920943|
|  Film-Noir|3.5483508757184237|
|     Horror| 2.730157352996138|
|    Musical| 3.376423123539873|
|    Mystery| 3.336813932383023|
|    Romance|3.2440490301647995|
|     Sci-Fi|3.1654460532512894|
|   Thriller| 3.136692475563836|
|        War| 3.489185428943569|
|    Western| 3.185617473056144|
|    unknown|2.2222222222222223|
+-----------+------------------+



In [ ]:
# Вариант_3 - решение в Pandas
df_result = spark_movie_rating.select("*").toPandas()  # переносим датафрейм из Spark в Pandas

df_result = df_result.melt('avg(rating)').rename(columns = {'variable' : 'genre'})

df_result['avg_rating'] = df_result['value'] * df_result['avg(rating)']

df_result = df_result[ df_result['avg_rating'] != 0 ]
df_result = df_result.groupby('genre'). agg({'avg_rating' : 'mean'}).reset_index()

df_result.sort_values('genre', ascending=True).head()

,genre,avg_rating
0,Action,2.966332
1,Adventure,3.143967
2,Animation,3.298813
3,Children,2.916885
4,Comedy,3.000565


In [ ]:
# Задание_11
# Spark - получить 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами (по количеству оценок, либо по самой оценке - на Ваш выбор)

# Top-5 популярных фильмов (самый большой рейтинг)

df_top_sum_rating = spark_rating.groupBy('film_id').sum('rating').sort('sum(rating)', ascending = False).limit(5)

df_top_sum_rating.show()

+-------+-----------+
|film_id|sum(rating)|
+-------+-----------+
|     50|       2541|
|    100|       2111|
|    181|       2032|
|    258|       1936|
|    174|       1786|
+-------+-----------+



In [ ]:
# Top-5 самых непопулярными фильмов (малое количество оценок)

df_top_count_rating = spark_rating.groupBy('film_id').count().sort('count', ascending = True).limit(5)

df_top_count_rating.show()

+-------+-----+
|film_id|count|
+-------+-----+
|   1507|    1|
|   1650|    1|
|   1460|    1|
|   1339|    1|
|   1580|    1|
+-------+-----+

